## Notebook to check VTK functionality

In [3]:
import vtk
import numpy as np
import random
from vtk.util.numpy_support import vtk_to_numpy
from math import cos, sin, pi

#Load data into VTK table object (https://github.com/Kitware/VTK/blob/master/Examples/Infovis/Python/tables3.py)
csv_source = vtk.vtkDelimitedTextReader()
csv_source.SetFieldDelimiterCharacters(",")
csv_source.SetHaveHeaders(False)
csv_source.SetDetectNumericColumns(True)
csv_source.SetFileName('/home/ntolley/Jones_Lab/lfp_reeb_local/data/gbarEvPyrAmpa_sweep/points/gbarEvPyrAmpa_sweep1.csv')
csv_source.Update()

T = csv_source.GetOutput()

# USER: vtkReebGraph's Build() function will rely on a vtkPolyData that
#  specifically only has triangles.  vtkUnstructuredGrid could be used
#  for tetrahedra.  
#  See http://www.vtk.org/doc/nightly/html/classvtkReebGraph.html#details
grid = vtk.vtkPolyData()

points = vtk.vtkPoints()
data = vtk.vtkFloatArray()
data.SetNumberOfComponents(1)
data.SetName("Function Value")

points.SetNumberOfPoints(T.GetNumberOfRows())
for r in range(T.GetNumberOfRows()):
    points.InsertPoint(r, T.GetColumn(0).GetValue(r),T.GetColumn(1).GetValue(r),T.GetColumn(2).GetValue(r))
    data.InsertNextValue(T.GetColumn(2).GetValue(r))
    

grid.SetPoints(points)
grid.GetPointData().SetScalars(data)

point_bounds = points.GetBounds()
x, y = int(point_bounds[1]), int(point_bounds[3])


tris = vtk.vtkCellArray()

for j in range(y) :
   for i in range(x) :
      tri = [j*x+i, j*x+i+1, (j+1)*x+i]
      tris.InsertNextCell(3, tri)
      
      tri = [j*x+i+1, (j+1)*x+i, (j+1)*x+i+1]
      tris.InsertNextCell(3, tri)
   #
#

grid.SetPolys(tris)

In [4]:
decimate = vtk.vtkDecimatePro()
decimate.SetInputData(grid)
decimate.SetTargetReduction(0.99)
decimate.PreserveTopologyOn()
decimate.Update()

decimated = vtk.vtkPolyData()
decimated.ShallowCopy(decimate.GetOutput())


data_decimated = vtk.vtkFloatArray()
data_decimated.SetNumberOfComponents(1)
data_decimated.SetName("Function Value")

# for r in range(decimated.)

# data_decimated.InsertNextValue(T.GetColumn(2).GetValue(r))


In [5]:
reeb_graph = vtk.vtkReebGraph()
err = reeb_graph.Build(decimated, decimated.GetPointData().GetScalars())

In [6]:
# USER: Reeb Graph does not (seem to) pass the data array around, it
#  only stores the Vertex Ids in the graph that's constructed.  The next
# #  few lines add it back for coloring in the vtkGraphLayoutView
data2 = vtk.vtkFloatArray()
data2.SetNumberOfComponents(1)
data2.SetName("Function Value")
data3 = vtk.vtkFloatArray()
data3.SetNumberOfComponents(1)
data3.SetName("Glyph Size")

info = reeb_graph.GetVertexData().GetAbstractArray("Vertex Ids")
for i in range(info.GetNumberOfTuples()) :
   # USER: uncomment the following to see data values in the Reeb graph
   print(info.GetTuple(i), data.GetTuple(int(info.GetTuple(i)[0])))
   data2.InsertNextValue(data.GetTuple(int(info.GetTuple(i)[0]))[0])
   # USER: this controls the scale / size of glyphs in graph
   data3.InsertNextValue(4.0)

# reeb_graph.GetVertexData().AddArray(data2)
# reeb_graph.GetVertexData().AddArray(data3)

(0.0,) (-8.466744475299492e-06,)
(8.0,) (-9.703555406304076e-05,)
(1.0,) (-8.629282092442736e-05,)
(10.0,) (-9.939880692400038e-05,)
(12.0,) (-0.00010029145778389648,)
(14.0,) (-0.00010041619680123404,)
(3.0,) (-7.385455683106557e-05,)
(15.0,) (-0.00010033490980276838,)
(5.0,) (-8.760193304624408e-05,)
(6.0,) (-9.18855657801032e-05,)
(18.0,) (-9.992032573791221e-05,)
(20.0,) (-9.961205796571448e-05,)
(22.0,) (-9.935373964253813e-05,)
(24.0,) (-9.914776455843821e-05,)
(26.0,) (-9.898834832711145e-05,)
(28.0,) (-9.888044587569311e-05,)
(30.0,) (-9.880027209874243e-05,)
(31.0,) (-9.877366392174736e-05,)
(34.0,) (-9.871595102595165e-05,)
(36.0,) (-9.869411587715149e-05,)
(268.0,) (-0.004611171782016754,)
(40.0,) (-9.865401079878211e-05,)
(42.0,) (-9.862695151241496e-05,)
(44.0,) (-9.860472346190363e-05,)
(47.0,) (-9.855566895566881e-05,)
(50.0,) (-9.850666538113728e-05,)
(51.0,) (-9.848534682532772e-05,)
(52.0,) (-9.84653306659311e-05,)
(53.0,) (-9.844722808338702e-05,)
(54.0,) (-9.8425414

In [7]:
num_tuples = reeb_graph.GetVertexData().GetAbstractArray(0).GetNumberOfTuples()
vert_id = []
for i in range(num_tuples):
    vert_id.append(reeb_graph.GetVertexData().GetAbstractArray(0).GetTuple(i)[0])




In [101]:
# z = reeb_graph.GetEdgeData().GetAbstractArray(0)
# print(reeb_graph.GetEdgeData().GetAbstractArray(0).GetValue(0).GetTypeAsString())
# print(reeb_graph.GetEdgeData().GetAbstractArray(0).GetDataTypeAsString())
# print(reeb_graph.GetEdgeData().GetAbstractArray(0).GetValue(1).ToArray().GetTuple(1))

edge_data = reeb_graph.GetEdgeData().GetAbstractArray(0)

edge_count = 0

num_edges = reeb_graph.GetNumberOfEdges()
edge_list = []
for e in range(num_edges):
    vertex_neighbors = edge_data.GetValue(e).ToArray()
    num_neighbors = vertex_neighbors.GetNumberOfTuples()

    neighbors = []
    for n in range(num_neighbors):
        neighbors.append(vertex_neighbors.GetTuple(n)[0])

    edge_list.append(neighbors)




In [109]:
print(len(edge_list))


5158


AttributeError: 'list' object has no attribute 'size'